# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 1

In [2]:
! pip install beautifulsoup4
! pip install lxml
! pip install html5lib
! pip install requests

import pandas as pd
import html
from bs4 import BeautifulSoup
import requests
import urllib.request


In [3]:
!wget -q -O 'Toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [146]:
with open('Toronto_data.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [147]:
#table = soup.find('table',class_='wikitable sortable')
table = soup.find('table')
#sum = table.td.text
#print(table)

In [148]:
column_names = ['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [149]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [150]:
df=df[df.Borough != 'Not assigned']
#df=df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [151]:
#df[df.Neighbourhood =='Not assigned']=df.Borough
df[df['Neighbourhood']=='Not assigned']=df['Borough']

In [152]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [153]:
temp_df=df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [154]:
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [155]:
df_merge = pd.merge(df, temp_df, on='Postcode')

In [156]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [157]:
df_merge.drop_duplicates(inplace=True)

In [158]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [159]:
df_merge.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [160]:
df_merge.shape

(103, 3)

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 2

In [141]:
!wget -q -O 'geo_data.csv' http://cocl.us/Geospatial_data

In [161]:
df_can = pd.read_csv('geo_data.csv')

In [162]:
df_can.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [168]:
df_geo = df_can.rename(columns={"Postal Code": "Postcode"})
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [169]:
df_joined = pd.merge(df, df_geo, on='Postcode')

In [170]:
# Merged Dataframe 
df_joined.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
